# **Trabajo Final**

# Trabajo Pratico 1 *Extraccion de datos*

En los siguientes codigo se va a extraer de dos APIs diferentes. Una de las apis se va a generar un codigo para una extraccion FULL y la otra api se va a generar un codigo para una extraccion incremental

Importo las libreria que se van a utilizar durante todo el codigo

In [2]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta

En el siguiente codigo se encuentra 2 funciones que se encargan de:


*   get_data:
        Realizar una solicitud GET a una API para obtener datos. y retornar el JSON de "data" o un codigo de error si eso sucede
*   build_table
        Construir un DataFrame de pandas a partir de datos en formato JSON. y retornar el mismo

In [5]:
def get_data(base_url, endpoint, params=None):
    """
    Realiza una solicitud GET a una API para obtener datos.

    :param base_url: La URL base de la API.
    :param endpoint: El endpoint (ruta) de la API para obtener datos específicos.
    :param params: Los parámetros de la solicitud GET.
    :return: Los datos obtenidos de la API en formato JSON.
    """
    try:
        endpoint_url = f"{base_url}/{endpoint}"
        response = requests.get(endpoint_url, params=params)
        response.raise_for_status()  # Levanta una excepción si hay un error en la respuesta HTTP.

        # Verificar si los datos están en formato JSON.
        try:
            data = response.json()
            data = data["data"]
        except:
            print("El formato de respuesta no es el esperado")
            return None
        return data

    except requests.exceptions.RequestException as e:
        # Capturar cualquier error de solicitud, como errores HTTP.
        print(f"La petición ha fallado. Código de error : {e}")
        return None

def build_table(json_data):
    """
    Construye un DataFrame de pandas a partir de datos en formato JSON.

    :param json_data: Los datos en formato JSON obtenidos de la API.
    :return: Un DataFrame de pandas con los datos estructurados.
    """
    df = pd.json_normalize(json_data)
    return df

def save_to_parquet(df, save_path, partition_col=None):
    """
    Guarda un DataFrame en formato Parquet en la ubicación especificada.

    Args:
        df (pd.DataFrame): El DataFrame que se desea guardar.
        save_path (str): La ruta donde se guardará el archivo Parquet.
        partition_col (str or list, optional): Columna(s) por la cual particionar los datos en el formato Parquet.

    Raises:
        TypeError: Si el argumento df no es un DataFrame de pandas.
        ValueError: Si partition_col no es None ni una cadena o lista de cadenas.
    """
    # Verificar que df sea un DataFrame
    if not isinstance(df, pd.DataFrame):
        raise TypeError("El argumento 'df' debe ser un DataFrame de pandas.")

    # Verificar el tipo de partition_col
    if partition_col is not None and not isinstance(partition_col, (str, list)):
        raise ValueError("El argumento 'partition_col' debe ser un string o una lista de strings.")

    # Crear el directorio si no existe
    directory = os.path.dirname(save_path)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)

    try:
        # Guardar el DataFrame en formato Parquet
        df.to_parquet(save_path, partition_cols=partition_col)
        print("DataFrame guardado exitosamente en formato Parquet.")
    except Exception as e:
        print(f"Error al guardar el DataFrame en formato Parquet: {str(e)}")



EXTRACCION DE DATOS

Extraccion de datos de manera FULL

El siguiente codigo extrae datos de una API la cual nos brinda datos de criptomonedas. Se realiza una extraccion FULL ya que extraemos todos los datos disponible del mismo

La siguiente extraccion se realizo en https://docs.coincap.io/#d8fd6001-e127-448d-aadd-bfbfe2c89dbe

In [6]:
url_base = "https://api.coincap.io/v2" # Coloco la url base para su luego utilizacion
endpoint = "assets" # Coloco el endpoint para su luego utilizacion
#Obtengo los datos
assets = get_data(url_base, endpoint)
if assets: # Pregunto si existe y genero el data frame del diccionario extraido
  df_assets = build_table(assets)
df_assets.head(70) # Imprimo 10 datos obtenidos anteriormente

,id,rank,symbol,name,supply,maxSupply,marketCapUsd,volumeUsd24Hr,priceUsd,changePercent24Hr,vwap24Hr,explorer
0,bitcoin,1,BTC,Bitcoin,19477281.0000000000000000,21000000.0000000000000000,500994752507.1261839121789984,2715836217.4731576395504195,25722.0067065380524064,-0.0487906023433916,25729.4067287557279717,https://blockchain.info/
1,ethereum,2,ETH,Ethereum,120218193.7760083100000000,None,196012666883.1797416603251611,1520488447.0589573385291366,1630.4742296194569339,0.3451598999123823,1628.7008637162661338,https://etherscan.io/
2,tether,3,USDT,Tether,82905516464.3926200000000000,None,82873616106.2361729445093061,6450341705.6572308371713923,0.9996152203192637,-0.0297574248884655,0.9993173433225742,https://www.omniexplorer.info/asset/31
3,binance-coin,4,BNB,BNB,166801148.0000000000000000,166801148.0000000000000000,35758417257.1759647816372904,86716349.1096365628651110,214.3775248907517398,-0.1846379260511081,214.7816799617335794,https://etherscan.io/token/0xB8c77482e45F1F44d...
4,usd-coin,5,USDC,USD Coin,25995360823.7995100000000000,None,25997594719.1697846376563055,415526506.9643971011926852,1.0000859343859628,-0.0060881816707190,1.0000172587576418,https://etherscan.io/token/0xa0b86991c6218b36c...
...,...,...,...,...,...,...,...,...,...,...,...,...
65,frax-share,66,FXS,Frax Share,73523518.6128336300000000,None,400810044.8877325246960251,234979.5235563909322332,5.4514535273855975,-1.7155368108373561,5.5220364321200416,https://etherscan.io/token/0x3432b6a60d23ca0df...
66,casper,67,CSPR,Casper,11298380822.0000000000000000,None,393693070.1995247491435850,830465.4342197094912781,0.0348450876636175,-0.0167758078339930,0.0350322776363729,https://casperstats.io/
67,huobi-token,68,HT,Huobi Token,162233844.0000000000000000,500000000.0000000000000000,385693278.5866639687578084,7627585.5579876250841704,2.3773909874604461,0.3929160120473716,2.3678957145262417,https://etherscan.io/token/0x6f259637dcd74c767...
68,curve-dao-token,69,CRV,Curve DAO Token,867670568.0000000000000000,3303030299.0000000000000000,384334840.1172363844682856,3133064.8735919561540198,0.4429501867317417,1.9303134810123970,0.4399238456976008,https://etherscan.io/token/0xD533a949740bb3306...


Extraccion de datos de manera incremental

El siguiente codigo extrae datos de concentracion de una 'formula' y lugar especificado. La extraccion se genera de forma incremental por cada ejecucion del mismo. Generando nuevos resultados por cada hora transcurrida

La extraccion se realizo en https://api-docs.luchtmeetnet.nl/

In [ ]:
endpoint = "concentrations"
base_url = "https://api.luchtmeetnet.nl/open_api"
start_date = datetime.utcnow() - timedelta(hours=1)

end_date = start_date.strftime("%Y-%m-%dT%H:59:59Z") # Guardo la fecha actual una hora adelantada en el formato requerido por el Provedor del API
start_date = start_date.strftime("%Y-%m-%dT%H:00:00Z") # Guardo la fecha actual en el formato requerido por el Provedor del API

# Creo un diccionario Params para guardar los parametros requeridos por el API
params = {
    "formula": "no2",
    "longitude": "4.458807",
    "latitude": "51.924452",
    "start": start_date,
    "end": end_date
    }

Concentrations = get_data(base_url, endpoint,params) # Obtengo los datos en formato JSON

df_Concentrations = build_table(Concentrations) # Le doy un formato mas legible a los datos obtenidos anteriormente

df_Concentrations.head() # Imprimo el resultado

El siguiente codigo extrae datos de una API la cual nos brinda datos de criptomonedas. Se realiza una extraccion incremental.

Se va a extraer los datos de donde se generan los intercambios de las monedas y cuanto representa esto en el mercado del mismo.

La siguiente extraccion se realizo en https://docs.coincap.io/#d8fd6001-e127-448d-aadd-bfbfe2c89dbe

In [24]:
url_base = "https://api.coincap.io/v2" # Coloco la url base para su luego utilizacion
endpoint = "exchanges" # Coloco el endpoint para su luego utilizacion
#Obtengo los datos
exchanges = get_data(url_base, endpoint)
if exchanges: # Pregunto si existe y genero el data frame del diccionario extraido
  df_exchanges = build_table(exchanges)
df_exchanges.head(5) # Imprimo 10 datos obtenidos anteriormente

,exchangeId,name,rank,percentTotalVolume,volumeUsd,tradingPairs,socket,exchangeUrl,updated
0,binance,Binance,1,49.002643480933881365000000000000000000,3870363025.9556094648391777,833,True,https://www.binance.com/,1693877950682
1,digifinex,DigiFinex,2,6.229106351076280273000000000000000000,491991884.3833654146176806,143,False,https://www.digifinex.com/,1693877947582
2,gdax,Coinbase Pro,3,5.767688306368357697000000000000000000,455547823.1473400008887099,298,True,https://pro.coinbase.com/,1693877949865
3,kraken,Kraken,4,4.561290765257274448000000000000000000,360263240.3974970033152806,343,False,https://kraken.com,1693877949184
4,gate,Gate,5,3.735963562402067499000000000000000000,295076636.9137683467528058,1284,False,https://gate.io/,1693877950625


El siguiente codigo extrae datos de una API la cual nos brinda datos de criptomonedas. Se realiza una extraccion incremental.

Se va a extraer los datos del mercado de criptomonedas.

La siguiente extraccion se realizo en https://docs.coincap.io/#d8fd6001-e127-448d-aadd-bfbfe2c89dbe

In [7]:
url_base = "https://api.coincap.io/v2" # Coloco la url base para su luego utilizacion
endpoint = "markets" # Coloco el endpoint para su luego utilizacion
#Obtengo los datos
markets = get_data(url_base, endpoint)
if markets: # Pregunto si existe y genero el data frame del diccionario extraido
  df_markets = build_table(markets)
df_markets.head(10) # Imprimo 10 datos obtenidos anteriormente

,exchangeId,rank,baseSymbol,baseId,quoteSymbol,quoteId,priceQuote,priceUsd,volumeUsd24Hr,percentExchangeVolume,tradesCount24Hr,updated
0,alterdice,1,ETH,ethereum,BTC,bitcoin,0.0638590000000000,1642.7457635199827798,0.0000000000000000,None,55,1693950108081
1,alterdice,2,ZRX,0x,BTC,bitcoin,0.0000066700000000,0.1715829286815999,0.0000000000000000,None,919,1693950118160
2,alterdice,3,WAVES,waves,ETH,ethereum,0.0009400000000000,1.5329123960408165,0.0000000000000000,None,24,1693950070615
3,alterdice,4,LINK,chainlink,ETH,ethereum,0.0037200000000000,6.0664192694381250,0.0000000000000000,None,60,1693950119943
4,alterdice,5,DASH,dash,USDT,tether,26.5300000000000000,26.5198045473220844,0.0000000000000000,None,62,1693950056183
5,alterdice,6,BNB,binance-coin,ETH,ethereum,0.1262010000000000,205.8032737156883904,0.0000000000000000,None,14,1693950113413
6,alterdice,7,LTC,litecoin,USDT,tether,66.2300000000000000,66.2045478767109555,0.0000000000000000,None,198,1693950055002
7,alterdice,8,ETH,ethereum,USDT,tether,1665.1900000000000000,1664.5500691351399060,0.0000000000000000,None,305,1693950053825
8,alterdice,9,WAVES,waves,USDT,tether,1.5511000000000000,1.5505039138089440,0.0000000000000000,None,126,1693950071241
9,alterdice,10,WAVES,waves,BTC,bitcoin,0.0000594000000000,1.5280398746157468,0.0000000000000000,None,67,1693950070020


#Trabajo Practico 2 **Almacenamiento**

In [26]:
!pip install -q fastparquet # Instalo FastParquet para agilizar los tiempos

ERROR: Invalid requirement: '#'


In [8]:
def crear_directorio(dir_nuevo):
   directory = os.path.dirname(dir_nuevo) #Obtengo el directorio de dir_nuevo
   if directory and not os.path.exists(directory): #Pregunto si existe y si no es una cadena vacia, si cumple la condicion lo creo en caso contrario lo ignoro
        os.makedirs(directory) # Creo el directorio si cumple las condiciones

In [16]:
dir_coincap_assets = "datalake_Santi/raw_date/coincap/assets/"
dir_mediciones = "datalake_Santi/raw_date/luchtmeetnet/concentrations/N02/"
dir_coincap_exchanges = "datalake_Santi/raw_date/coincap/exchanges/"
dir_coincap_markets = "datalake_Santi/raw_date/coincap/markets/"
#Creo la ruta donde se van a almacenar los datos extraidos anteriormente
crear_directorio(dir_coincap_assets)
crear_directorio(dir_mediciones)
crear_directorio(dir_coincap_exchanges)
crear_directorio(dir_coincap_markets)

## A continuacion introduzco los data frames extraidos anteriormente en sus respectivos directorios con un formato parquet particionados segun cada uno.

In [29]:
# Guardo los data frames con formato parquet almacenandolos en una carpeta sin hacer particiones
save_to_parquet(df_assets,"datalake_Santi/raw_date/coincap/assets/assets.parquet")
df_assets_pq = pd.read_parquet("datalake_Santi/raw_date/coincap/assets/assets.parquet")

DataFrame guardado exitosamente en formato Parquet.


In [30]:
df_Concentrations["timestamp_measured"] = pd.to_datetime(df_Concentrations.timestamp_measured)
df_Concentrations["fecha"] = df_Concentrations.timestamp_measured.dt.date
df_Concentrations["hora"] = df_Concentrations.timestamp_measured.dt.hour
#Creo un nuevo data Frame con hora y fecha divido para luego guardarlo como un archivo con formato Parquet y particionarlo por fecha y hora
# Creo por cada fecha y hora una carpeta donde guardo los datos de los correspondientes momentos
save_to_parquet(df_Concentrations,"datalake_Santi/raw_date/luchtmeetnet/concentrations/N02",["fecha"])
df_Concentrations_pq = pd.read_parquet("datalake_Santi/raw_date/luchtmeetnet/concentrations/N02")
df_Concentrations.head()

DataFrame guardado exitosamente en formato Parquet.


,formula,value,timestamp_measured,fecha,hora
0,NO2,30.0,2023-09-05 01:00:00+00:00,2023-09-05,1
1,NO2,30.0,2023-09-05 02:00:00+00:00,2023-09-05,2
2,NO2,20.0,2023-09-05 03:00:00+00:00,2023-09-05,3
3,NO2,20.0,2023-09-05 04:00:00+00:00,2023-09-05,4
4,NO2,20.0,2023-09-05 05:00:00+00:00,2023-09-05,5


In [67]:
#Creo un nuevo data Frame con fecha divido para luego guardarlo como un archivo con formato Parquet y particionarlo por fecha
df_exchanges['updated'] = pd.to_datetime(df_exchanges['updated'], unit = 'ms') #Cambio el formato UNIX timestamp a formato un formato mas estandar
df_exchanges['updated'] = pd.to_datetime(df_exchanges.updated)
df_exchanges["fecha"] = df_exchanges.updated.dt.date
save_to_parquet(df_exchanges,"datalake_Santi/raw_date/coincap/exchanges",["fecha"])
#df_exchanges.head(5)
df_exchanges_pq = pd.read_parquet("datalake_Santi/raw_date/coincap/exchanges")

DataFrame guardado exitosamente en formato Parquet.


In [30]:
df_markets['updated'] = pd.to_datetime(df_markets['updated'], unit = 'ms')
df_markets['updated'] = pd.to_datetime(df_markets["updated"])
df_markets["fecha"] = df_markets.updated.dt.date
save_to_parquet(df_markets,"datalake_Santi/raw_date/coincap/markets",["fecha"])
df_markets = pd.read_parquet("datalake_Santi/raw_date/coincap/markets")
df_markets

DataFrame guardado exitosamente en formato Parquet.


,exchangeId,rank,baseSymbol,baseId,quoteSymbol,quoteId,priceQuote,priceUsd,volumeUsd24Hr,percentExchangeVolume,tradesCount24Hr,updated,fecha
0,alterdice,1,ETH,ethereum,BTC,bitcoin,0.0638590000000000,1642.7457635199827798,0.0000000000000000,None,55,2023-09-05 21:41:48.081,2023-09-05
1,alterdice,2,ZRX,0x,BTC,bitcoin,0.0000066700000000,0.1715829286815999,0.0000000000000000,None,919,2023-09-05 21:41:58.160,2023-09-05
2,alterdice,3,WAVES,waves,ETH,ethereum,0.0009400000000000,1.5329123960408165,0.0000000000000000,None,24,2023-09-05 21:41:10.615,2023-09-05
3,alterdice,4,LINK,chainlink,ETH,ethereum,0.0037200000000000,6.0664192694381250,0.0000000000000000,None,60,2023-09-05 21:41:59.943,2023-09-05
4,alterdice,5,DASH,dash,USDT,tether,26.5300000000000000,26.5198045473220844,0.0000000000000000,None,62,2023-09-05 21:40:56.183,2023-09-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,bibox,41,JST,just,USDT,tether,0.0204810000000000,0.0204731291720205,1117726.4455460288249868,0.5877894957531997,9784,2023-09-05 21:33:56.429,2023-09-05
96,bibox,42,HT,huobi-token,USDT,tether,2.3800000000000000,2.3790853683613479,1096563.1620418085830399,0.5766601574531197,4111,2023-09-05 21:28:07.253,2023-09-05
97,bibox,43,FTM,fantom,USDT,tether,0.2024120000000000,0.2023342132692257,1012305.0323466977397917,0.5323505289532746,7378,2023-09-05 21:28:23.708,2023-09-05
98,bibox,44,QTUM,qtum,USDT,tether,2.1760000000000000,2.1751637653589467,921744.2921622321986105,0.4847264863977999,4133,2023-09-05 21:27:38.968,2023-09-05
